In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive//NLP Final Project
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1XAIHXAym6w3aZynQYubNhZ-kR2ByPq8C/NLP Final Project
 BERT.ipynb                                 sres.gsheet
 Data/                                      test.csv
 data_preprocessing.ipynb                   Testing_Bill_Parsing-Garrett.ipynb
 distilBERT_20.pth                          test.ipynb
 DistilBERT_Model.ipynb                     train_18-22.csv
 finetuning-sentiment-model-3000-samples/   train2.csv
 GPT.csv                                    train.csv
 GPT.gsheet                                 train.gsheet
'GPT Input.gdoc'                            val_18-22.csv
 GPT.ipynb                                  val2.csv
 hres.csv                                   val2.gsheet
 KeyBERT.ipynb                              val.csv
 Lisa_count_tf-idf.ipynb                    val.gsheet
 sbill.csv                           

In [ ]:
%pip install keybert

In [ ]:
import random

import pandas as pd
import numpy as np
from collections import OrderedDict
import re

from sklearn.utils import shuffle
import string
from transformers import DistilBertTokenizer, DistilBertModel
import torch
import time
from datetime import datetime
import ast
from torch.nn.utils.rnn import pad_sequence
from keybert import KeyBERT

from tqdm.notebook import tqdm

In [ ]:
kwd_model = KeyBERT()
bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
def bert_encoder(text):
    """ Compute semantic vector with BERT
    Parameters
    ----------
    seq: string to encode

    Returns
    -------
        np array
    """
    words = text.split(" ")
    # words = [word for word in words if word in bert_tokenizer.vocab.keys()]
    if len(words) > 2048:
        words = words[:2048]
    n_words = int(np.log2(len(words)))
    words = " ".join(words)
    keywords = kwd_model.extract_keywords(words, keyphrase_ngram_range=(1, 3), top_n=n_words)
    # keywords2 = kwd_model.extract_keywords(words, keyphrase_ngram_range=(2, 2), top_n=n_words)
    # keywords3 = kwd_model.extract_keywords(words, keyphrase_ngram_range=(3, 3), top_n=n_words)
    keywords = [word[0] for word in keywords]
    # keywords = list(set(keywords))
    s = " ".join(keywords)
    s = s + " " + text
    tokens = bert_tokenizer(s, return_tensors='pt', padding=True, max_length=64, truncation=True)
    return tokens

In [ ]:
def clean(seq):
    """ Preprocess sentences for BERT
    Parameters
    ----------
    seq: str, raw sentence

    Returns
    -------
    str, preprocessed sentence
    """
    seq = re.sub('\]|\[|\)|\(|\=|\,|\;', ' ', seq)
    seq = " ".join([word.lower() if word.isupper() else word for word in seq.strip().split()])
    seq = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', seq))
    seq = " ".join([word for word in seq.split() if not bool(re.search(r'\d', word))])
    table = str.maketrans(dict.fromkeys(list(string.punctuation)))
    content = seq.translate(table)
    seq = " ".join([word.lower().strip() for word in content.strip().split()])
    return seq

In [ ]:
def load_data(train, validation):
    """ Load HDFS unstructured log into train and test data
    Arguments
    ---------
        train: str, the file path of training resolutions.
        validation: str, the file path of validation resolutions.
    Returns
    -------
        (x_train, y_train): the training data
        (x_val, y_val): the validation data
    """
    encoder = bert_encoder

    #get data
    train_data = pd.read_csv(train)
    validation = pd.read_csv(validation)

    #convert training data into numpy array
    X_train = train_data['text']
    X_train = X_train.apply(ast.literal_eval)
    X_train = np.array(X_train)
    y_train = np.array(train_data['ideology'].tolist())

    #filter entries with no assigned ideology
    train_inds = np.where(np.isnan(y_train))[0]
    mask = np.ones(len(y_train), dtype=bool)
    mask[train_inds] = False
    y_train = y_train[mask]
    X_train = X_train[mask]

    #convert validatioin data into numpy array
    X_val = validation['text']
    X_val = X_val.apply(ast.literal_eval)
    X_val = np.array(X_val)
    y_val = np.array(validation['ideology'].tolist())

    #filter entries with no assigned ideology
    val_inds = np.where(np.isnan(y_val))[0]
    mask = np.ones(len(y_val), dtype=bool)
    mask[val_inds] = False
    y_val = y_val[mask]
    X_val = X_val[mask]

    #convert labels into binary encoding
    # y_train = np.around(y_train).astype(float)
    # y_val = np.around(y_val).astype(float)

    return X_train, y_train, X_val, y_val

In [ ]:
X_train, y_t, X_val, y_v = load_data('train.csv', 'val.csv')

In [ ]:
X_t = []
X_t_mask = []
#iterate through, clean, and tokenize X_train
for i, text in enumerate(X_train):
    seq = " ".join(text)
    s = seq.lower()
    token = bert_encoder(s)
    X_t.append(token['input_ids'].squeeze())
    X_t_mask.append(token['attention_mask'].squeeze())
    if i % 100 == 0:
        print(i)

X_v = []
X_v_mask = []
#iterate through, clean, and tokenize X_val
for i, text in enumerate(X_val):
    seq = " ".join(text)
    s = seq.lower()
    token = bert_encoder(s)
    X_v.append(token['input_ids'].squeeze())
    X_v_mask.append(token['attention_mask'].squeeze())

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [ ]:
#pad list of tensors to the same size so they can be tensorized
X_t = pad_sequence(X_t, batch_first=True, padding_value=0)
X_t_mask = pad_sequence(X_t_mask, batch_first=True, padding_value=0)

X_v = pad_sequence(X_v, batch_first=True, padding_value=0)
X_v_mask = pad_sequence(X_v_mask, batch_first=True, padding_value=0)

In [ ]:
#convert list of tokens to tensors
X_train = torch.tensor(X_t)
X_val = torch.tensor(X_v)

X_train_mask = torch.tensor(X_t_mask)
X_val_mask = torch.tensor(X_v_mask)

<ipython-input-62-f74968e82929>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_t)
<ipython-input-62-f74968e82929>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_v)
<ipython-input-62-f74968e82929>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_mask = torch.tensor(X_t_mask)
<ipython-input-62-f74968e82929>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.

In [ ]:
#convert labels to tensors
y_train = torch.tensor(y_t)
y_val = torch.tensor(y_v)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import Dataset, TensorDataset, DataLoader
from sklearn.utils import shuffle

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, num_classes):
        super(TransformerClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768, num_classes)  # BERT base model output size is 768
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.mean(output.last_hidden_state, dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        logits = self.sigmoid(logits)
        return logits

In [ ]:
model = TransformerClassifier(num_classes=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

train_dataset = TensorDataset(X_train, X_train_mask, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, X_val_mask, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


for epoch in range(20):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, target = batch
        logits = model(input_ids, attention_mask)
        probs = logits[:,1].float()
        # target = torch.round(target)
        loss = loss_fn(probs, target.float())
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, target = batch
            logits = model(input_ids, attention_mask)
            probs = logits[:,1].float()
            # target = torch.round(target)
            val_loss += loss_fn(probs, target.float()).item()
            val_acc += (logits.argmax(dim=1) == np.around(target)).sum().item()
    val_loss /= len(val_loader)
    val_acc /= len(val_loader.dataset)
    print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1, Val Loss: 0.0591, Val Acc: 0.6881
Epoch 2, Val Loss: 0.0544, Val Acc: 0.7464
Epoch 3, Val Loss: 0.0581, Val Acc: 0.7360
Epoch 4, Val Loss: 0.0575, Val Acc: 0.7609
Epoch 5, Val Loss: 0.0551, Val Acc: 0.7630
Epoch 6, Val Loss: 0.0579, Val Acc: 0.7505
Epoch 7, Val Loss: 0.0543, Val Acc: 0.7630
Epoch 8, Val Loss: 0.0537, Val Acc: 0.7464
Epoch 9, Val Loss: 0.0555, Val Acc: 0.7505
Epoch 10, Val Loss: 0.0574, Val Acc: 0.7568
Epoch 11, Val Loss: 0.0578, Val Acc: 0.7505
Epoch 12, Val Loss: 0.0559, Val Acc: 0.7422
Epoch 13, Val Loss: 0.0580, Val Acc: 0.7443
Epoch 14, Val Loss: 0.0551, Val Acc: 0.7547
Epoch 15, Val Loss: 0.0563, Val Acc: 0.7464
Epoch 16, Val Loss: 0.0549, Val Acc: 0.7484
Epoch 17, Val Loss: 0.0566, Val Acc: 0.7505
Epoch 18, Val Loss: 0.0550, Val Acc: 0.7526
Epoch 19, Val Loss: 0.0555, Val Acc: 0.7484
Epoch 20, Val Loss: 0.0583, Val Acc: 0.7401


In [ ]:
def load_test_data(test):
    """ Load HDFS unstructured log into train and test data
    Arguments
    ---------
        test: str, the file path of test resolutions.
    Returns
    -------
        (x_test, y_test): the test data
    """
    encoder = bert_encoder

    #get data
    test_data = pd.read_csv(test)

    #convert training data into numpy array
    X_test = test_data['text']
    X_test = X_test.apply(ast.literal_eval)
    X_test = np.array(X_test)
    y_test = np.array(test_data['ideology'].tolist())

    #filter entries with no assigned ideology
    test_inds = np.where(np.isnan(y_test))[0]
    mask = np.ones(len(y_test), dtype=bool)
    mask[test_inds] = False
    y_test = y_test[mask]
    X_test = X_test[mask]

    #convert labels into binary encoding
    # y_train = np.around(y_train).astype(float)
    # y_val = np.around(y_val).astype(float)

    return X_test, y_test

In [ ]:
X_t, y_test = load_test_data('test.csv')

In [ ]:
X_test = []
X_test_mask = []
#iterate through, clean, and tokenize X_test
for i, text in enumerate(X_t):
    seq = " ".join(text)
    s = clean(seq).lower()
    token = bert_encoder(s)
    X_test.append(token['input_ids'].squeeze())
    X_test_mask.append(token['attention_mask'].squeeze())
    if i % 100 == 0:
        print(i)

0
100
200
300
400


In [ ]:
X_test = pad_sequence(X_test, batch_first=True, padding_value=0)
X_test_mask = pad_sequence(X_test_mask, batch_first=True, padding_value=0)

In [ ]:
X_test = torch.tensor(X_test)
X_test_mask = torch.tensor(X_test_mask)

<ipython-input-71-1c1b3c71ef1b>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test)
<ipython-input-71-1c1b3c71ef1b>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_mask = torch.tensor(X_test_mask)


In [ ]:
y_test = torch.tensor(y_test)

In [ ]:
test_dataset = TensorDataset(X_test, X_test_mask, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
preds = []
true = []
binary_preds = []
true_class = []
for batch in test_loader:
    input_ids, attention_mask, target = batch
    logits = model(input_ids, attention_mask)
    probs = logits[:,1].float()
    preds.extend(probs)
    true.extend(target)
    binary_preds.extend(logits.argmax(dim=1))
    true_class.extend(np.around(target))

In [ ]:
def tensors_to_list(list_of_tensors):
    list_of_arrays = [round(tensor.item(), 4) for tensor in list_of_tensors]
    return list_of_arrays


In [ ]:
preds = tensors_to_list(preds)
true = tensors_to_list(true)
binary_preds = tensors_to_list(binary_preds)
true_class = tensors_to_list(true_class)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_class, binary_preds))

              precision    recall  f1-score   support

         0.0       0.65      0.77      0.70       257
         1.0       0.57      0.41      0.48       186

    accuracy                           0.62       443
   macro avg       0.61      0.59      0.59       443
weighted avg       0.61      0.62      0.61       443



In [ ]:
comparison = pd.DataFrame(columns=['true', 'pred'])
comparison['true'] = true
comparison['pred'] = preds

In [ ]:
def mse(true, pred):
    return np.mean((true - pred)**2)

In [ ]:
mse(np.array(true), np.array(preds))

0.10106868501128669

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("val.csv")

# test_true = test['ideology'].values

train['ideology'].loc[train['ideology'] > .5] = 1 # convert to strict scores of 1
train['ideology'].loc[train['ideology'] <= .5] = 0

test['ideology'].loc[test['ideology'] > .5] = 1 # convert to strict scores of 1
test['ideology'].loc[test['ideology'] <= .5] = 0

train.dropna(axis='rows', inplace=True)

test.dropna(axis='rows', inplace=True)

<ipython-input-81-a7591d60ef8c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['ideology'].loc[train['ideology'] > .5] = 1 # convert to strict scores of 1
<ipython-input-81-a7591d60ef8c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['ideology'].loc[train['ideology'] <= .5] = 0
<ipython-input-81-a7591d60ef8c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ideology'].loc[test['ideology'] > .5] = 1 # convert to st

In [ ]:
X_train = train['text'].values
y_train = train['ideology'].values

X_test = test['text'].values
y_test = test['ideology'].values

In [ ]:
print(X_train[1])

['A resolution commemorating the 48th anniversary of the signing into law of the Employee Retirement Income Security Act of 1974, recognizing the importance the Employee Retirement Income Security Act of 1974 plays in protecting the interests of participants in employee benefit plans and their beneficiaries, and recognizing the need to encourage more workers to participate in such plans to save for their retirement.', 'Commemorating the 48th anniversary of the signing into law of the Employee Retirement Income Security Act of 1974, recognizing the importance the Employee Retirement Income Security Act of 1974 plays in protecting the interests of participants in employee benefit plans and their beneficiaries, and recognizing the need to encourage more workers to participate in such plans to save for their retirement.Whereas September 2, 2022, marks the 48th anniversary of the signing into law of the Employee Retirement Income Security Act of 1974 (29 U.S.C. 1001 et seq.) (referred to in

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
vec.fit(X_train)
x_train=vec.transform(X_train)
x_test=vec.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

score = lr.score(x_test, y_test)

print("LR score:", score)

LR score: 0.7837837837837838


In [ ]:
test2 = pd.read_csv("test.csv")
test2['ideology'].loc[test2['ideology'] > .5] = 1 # convert to strict scores of 1
test2['ideology'].loc[test2['ideology'] <= .5] = 0
X_test2 = test2['text'].values
y_test2 = test2['ideology'].values

<ipython-input-102-5f5f8e72fbb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['ideology'].loc[test2['ideology'] > .5] = 1 # convert to strict scores of 1
<ipython-input-102-5f5f8e72fbb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test2['ideology'].loc[test2['ideology'] <= .5] = 0


In [ ]:
x_test2=vec.transform(X_test2)

In [ ]:
preds2 = lr.predict(x_test2)

In [ ]:
mse(y_test2, preds2)

0.4018058690744921

In [ ]:
print(classification_report(y_test2, preds2))

              precision    recall  f1-score   support

         0.0       0.60      0.94      0.73       257
         1.0       0.60      0.13      0.21       186

    accuracy                           0.60       443
   macro avg       0.60      0.53      0.47       443
weighted avg       0.60      0.60      0.51       443



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(16, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])



In [ ]:
vec = CountVectorizer()
vec.fit(X_train)
x_train=vec.transform(X_train)
x_test=vec.transform(X_test)

model.summary()
model.fit(x_train, y_train)
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                756560    
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 756849 (2.89 MB)
Trainable params: 756849 (2.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
4/4 [==============================] - 0s 4ms/step - loss: 0.5464 - accuracy: 0.6632
test loss, test acc: [0.5464080572128296, 0.6632016897201538]


In [ ]:
test2 = pd.read_csv("test.csv")
# test2['ideology'].loc[test2['ideology'] > .5] = 1 # convert to strict scores of 1
# test2['ideology'].loc[test2['ideology'] <= .5] = 0
X_test2 = test2['text'].values
y_test2 = test2['ideology'].values

In [ ]:
x_test2=vec.transform(X_test2)

In [ ]:
preds2 = model.predict(x_test2)

14/14 [==============================] - 0s 2ms/step


In [ ]:
print(classification_report(y_test2, np.around(preds2)))

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
mse(y_test2, preds2)

0.10696556023019943